In [ ]:
!pip install tensorflow==2.16.1

In [ ]:
# Importing necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

# The Auto MPG Dataset

In [ ]:
# Get the data
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

# Load the dataset
raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
# Make a copy of the dataset
data = raw_dataset.copy()
data.head()

# Clean the data

In [ ]:
# Check the unknowns values
data.isna().sum()

In [ ]:
# Drop rows with missing values
data = data.dropna()

In [ ]:
# Map 'Origin' column to country names
data['Origin'] = data['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
data.head()

In [ ]:
# Convert categorical 'Origin' column to one-hot encoding
data = pd.get_dummies(data, columns=['Origin'], prefix='', prefix_sep='')
data.head()

# Split the data into training and test sets

In [ ]:
# Sample 80% of the data for training
train_dataset = data.sample(frac=0.8, random_state=0)

# Sample the remaining 20% for testing
test_dataset = data.drop(train_dataset.index)

# Inspect the data

In [ ]:
# Create pair plots of selected features
sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')

In [ ]:
# Display summary statistics of the training dataset
train_dataset.describe().transpose()

# Split features from labels

In [ ]:
# Separate the 'MPG' column as labels
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

# Normalization

In [ ]:
# Display mean and standard deviation of the training dataset
train_dataset.describe().transpose()[['mean', 'std']]

# The Normalization layer

In [ ]:
# Create a normalization layer
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
# Adapt the normalizer to the training features
normalizer.adapt(np.array(train_features))

In [ ]:
# Display the mean values after normalization
print(normalizer.mean.numpy())

In [ ]:
# Example of the first row before and after normalization
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

# Linear regression

In [ ]:
# Extract 'Horsepower' column as numpy array
horsepower = np.array(train_features['Horsepower'])

# Create a normalization layer for "Horsepower"
horsepower_normalizer = layers.Normalization(input_shape=[1,], axis=None)
horsepower_normalizer.adapt(horsepower)

In [ ]:
# Build a linear regression model for "Horsepower"
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

# Display model summary
horsepower_model.summary()

In [ ]:
# Predict the first 10 values of 'Horsepower'
horsepower_model.predict(horsepower[:10])

In [ ]:
# Compile the model with Adam optimizer and mean absolute error loss
horsepower_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = horsepower_model.fit(
    train_features['Horsepower'],
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
# Convert training history to DataFrame and display the last few rows
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
# Function to plot training loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
# Evaluate the model on the test set
test_results = {}

test_results['horsepower_model'] = horsepower_model.evaluate(
    test_features['Horsepower'],
    test_labels, verbose=0)

In [ ]:
# Generate predictions for a range of 'Horsepower' values
x = tf.linspace(0.0, 250, 251)
y = horsepower_model.predict(x)

In [ ]:
# Function to plot 'Horsepower' vs 'MPG' predictions
def plot_horsepower(x, y):
  plt.scatter(train_features['Horsepower'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()

In [ ]:
plot_horsepower(x, y)

# Linear regression with multiple inputs

In [ ]:
# Build a linear regression model with multiple inputs
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
# Predict the first 10 examples
linear_model.predict(train_features[:10])

In [ ]:
# Display the kernel (weights) of the dense layer
linear_model.layers[1].kernel

In [ ]:
# Compile the model
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
# Plot training loss
plot_loss(history)

In [ ]:
# Evaluate the model on the test set
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)

# Regression with a deep neural network (DNN)

In [ ]:
# Function to build and compile a DNN model
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

# Regression using a DNN and a single input

In [ ]:
# Build and compile a DNN model for 'Horsepower'
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

In [ ]:
# Display model summary
dnn_horsepower_model.summary()

In [ ]:
%%time
history = dnn_horsepower_model.fit(
    train_features['Horsepower'],
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
# Plot training loss
plot_loss(history)

In [ ]:
# Generate predictions for a range of 'Horsepower' values
x = tf.linspace(0.0, 250, 251)
y = dnn_horsepower_model.predict(x)

In [ ]:
# Plot 'Horsepower' vs 'MPG' predictions
plot_horsepower(x, y)

In [ ]:
# Evaluate the model on the test set
test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
    test_features['Horsepower'], test_labels,
    verbose=0)

# Regression using a DNN and multiple inputs

In [ ]:
# Build and compile a DNN model with multiple inputs
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
# Plot training loss
plot_loss(history)

In [ ]:
# Evaluate the model on the test set
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

# Performance

In [ ]:
# Display the test results as a DataFrame
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

# Make predictions

In [ ]:
# Predict MPG values for the test set
test_predictions = dnn_model.predict(test_features).flatten()

# Plot true vs predicted MPG values
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
# Plot prediction error histogram
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')

In [ ]:
# Save the DNN model
dnn_model.save('dnn_model.keras')

In [ ]:
# Load the saved model
reloaded = tf.keras.models.load_model('dnn_model.keras')

# Evaluate the reloaded model on the test set
test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
# Display the final test results
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T